# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [8]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set()

In [10]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [16]:
data.race.value_counts()

w    2435
b    2435
Name: race, dtype: int64

In [18]:
# number of callbacks for black-sounding names
callbacks_b = sum(data[data.race=='b'].call)

# number of callbacks for white-sounding names
callbacks_w = sum(data[data.race=='w'].call)

print('The number of callbacks for black-sounding names is ', callbacks_b, '.')
print('The number of callbacks for white-sounding names is ', callbacks_w, '.')

The number of callbacks for black-sounding names is  157.0 .
The number of callbacks for white-sounding names is  235.0 .


In [21]:
# callback proportion for 'b'

proportion_b = 157/2435

# callback proportion for 'w'

proportion_w = 235/2435

print('P_black_callback is: ', round(proportion_b*100,2), '%')
print('P_white_callback is: ', round(proportion_w*100,2), '%')


P_black_callback is:  6.45 %
P_white_callback is:  9.65 %


In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>


Q1. What test is appropriate for this problem? Does CLT apply?

A1. The test that is appropriate for this problem is a two-sample hypothesis test. For the CLT to apply, the samples must be **independent, normal,** and be **random**. To be independent, our sample size cannot be more than 10% of the population. Assuming that the population is the labor market in the United States, the sample size of 4870 (2435 each for 'w' and 'b') to be less than that value, therefore the experiment is **independent**. To be normal, our sample needs at least 10 success and 10 failures. As calculated above, there is more than 10 successes and failures for each population (resumes from 'w' and resumes from 'b'), therefore the experiment is **normal**. To be random, it needs to come froma randomized experiment. As stated in the problem statement, the researchers 'randomly assigned', therefore this experiment is **random**.

Q2. What are the null and alternate hypotheses? 

* H0 = There is no difference in the proportions of callbacks in regards to race => Pw - Pb = 0
* HA = There is a difference in the proportions of callbacks in regards to race | white-sounding names have a higher callback proportion => Pw > Pb

Not included in question, but I'll state the alpha value here for good practice:

alpha = 0.05 (5 %)

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

<div class="span5 alert alert-success">
<p> Your solution to Q3 here </p>
</div>

In [74]:
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data,func)

    return bs_replicates

def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

In [84]:
data_callback_c = data.call
data_callback_w = w.call
data_callback_b = b.call

In [108]:
# Bootstrap method

diff_props = diff_of_means(data_callback_w, data_callback_b)

# assume means are the same
data_callback_b_shifted = data_callback_b - np.mean(data_callback_b) + diff_props
data_callback_w_shifted = data_callback_w - np.mean(data_callback_w) + diff_props

# generate bootstrap replicates

bs_reps_b = draw_bs_reps(data_callback_b_shifted, np.mean, 10000)
bs_reps_w = draw_bs_reps(data_callback_w_shifted, np.mean, 10000)

diff_in_race = bs_reps_w - bs_reps_b

print('The margin of error of the sample is: ', round(np.std(diff_in_race),4))

print('The confidence interval for the difference in proportions is: -inf to ', round(np.percentile(diff_in_race,95),4))


p = np.sum(diff_in_race >= diff_props)/len(perm_replicates)
print('The p-value is: ', p)

The margin of error of the sample is:  0.0078
The confidence interval for the difference in proportions is: -inf to  0.0127
The p-value is:  0.0001


In [109]:
# Frequentist method


sample_size_b = len(data_callback_b)
sample_size_w = len(data_callback_w)

# assuming populations have the same proportions
var_c = np.var(data_callback_c)
std_combined = np.sqrt(var_c/sample_size_w + var_c/sample_size_b)

# print(p_diff)
# print(std_combined)

print('The margin of error of the sample is:', round(std_combined,4))

# the z-score for a one sided 95% confidence interval is 1.65
print('The confidence interval for the difference in proportions is: -inf to', round(1.65*std_combined,4))

z_score = diff_props/std_combined # 4.108

print('The p-value is: ', round(stats.norm.sf(z_score),4))

The margin of error of the sample is: 0.0078
The confidence interval for the difference in proportions is: -inf to 0.0129
The p-value is:  0.0


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>


Q4. Write a story describing the statistical significance in the context or the original problem.

A5. At an alpha value of 0.05, we can **reject** the null hypothesis that the proportions of 'black' and 'white' callbacks are equal. There is a strong suggestion to accept the alternative hypothesis. This basically suggests that there is a bias between the black-sounding names vs. white-sounding names in the experiment where the white-sounding names get significantly more callbacks. This may be a result of underlying racial bias of the employers.

Q5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

A5. This analysis does not necessarily mean that the race/name is the most important factor in callback success although it is suggested that having a white-sounding name may get preferential treatment. This is because there are other parameters that have a bigger impact than the race, e.g. experience, education, etc. Given that the experiment assigned identical resumes to the black and white-sounding names (i.e. controlling confounding variables), the analysis would probably stay the same. 

However, if the analysis was to figure out the most important factor in callback success rates, then I would explore the data more and look at the other features individually to see what may have an impact on callback success.